In [1]:
import os
import pandas as pd
import numpy as np
import requests

import pickle

import matplotlib.pyplot as plt
%matplotlib inline 

In [2]:
API_KEY = os.environ.get('SENATORS', None)
DATA_PATH = '../data/all-votes'

In [3]:
years = np.arange(1997, 2019, 1)
months = np.arange(1, 13, 1)

## Get general information about all voted laws for the last 20 years

In [4]:
laws = []
for y in years:
    for m in months:
        url = 'https://api.propublica.org/congress/v1/senate/votes/{}/{}.json'.format(y, m)
#         req = requests.get(url, headers={'X-API-Key': API_KEY}).json()
        results = req['results']['votes']
        for r in results:
            d = {}
            d['year'] = y
            d['month'] = m
            d['congress'] = r['congress']
            d['vote_uri'] = r['vote_uri']
            d['bill_api_uri'] = r['bill'].get('api_uri', None)
            d['vote_type'] = r['vote_type']
            d['result'] = r['result']
            d['date'] = r['date']
            d['total_yes'] = r['total']['yes']
            d['total_no'] = r['total']['no']
            d['total_not_voting'] = r['total']['not_voting']
            d['democrats_no'] = r['democratic']['no']
            d['democrats_yes'] = r['democratic']['yes']
            d['republican_no'] = r['republican']['no']
            d['republican_yes'] = r['republican']['yes']
            d['independent_no'] = r['independent']['no']
            d['independent_yes'] = r['independent']['yes']
            laws.append(d)

df = pd.DataFrame(laws)
print(df.shape)
df.to_csv(DATA_PATH + '/laws_20years.csv', index=False)

(6834, 16)


In [36]:
all_laws = pd.read_csv(DATA_PATH + '/laws_20years.csv')

## Get general information about congress members of the last 20 years

In [40]:
congresses_members = []
for congress in all_laws['congress'].unique():
    url = 'https://api.propublica.org/congress/v1/{}/senate/members.json'.format(congress)
#     req = requests.get(url, headers={'X-API-Key': API_KEY}).json()
    members = req['results'][0]['members']
    infos = [{'congress': congress, 
              'name': m['first_name'] + ' ' + m['last_name'],
              'id': m['id'], 
              'api_uri': m['api_uri'], 
              'party': m['party'],
              'gender': m['gender'], 
              'seniority': m['seniority'],
              'dw_nominate': m['dw_nominate']
             } for m in members]
    congresses_members += infos

df = pd.DataFrame(congresses_members)
print(df.shape)
df.to_csv(DATA_PATH + '/members_20years.csv', index=False)

(1133, 8)


In [8]:
all_members = pd.read_csv(DATA_PATH + '/members_20years.csv')

In [139]:
print(all_members.id.nunique())

231


## Build adjacency matrix for every single voted laws for the last 20 years

In [5]:
def get_adjacency(members, votes):
    n = len(members)
    adjancency = np.zeros((n, n))
    
    for i, i_id in enumerate(members):
        for j, j_id in enumerate(members):
            if i != j:
                vote_i = votes.loc[i_id]['vote_position'].lower() if i_id in votes.index else 'not voting'
                vote_j = votes.loc[j_id]['vote_position'].lower() if j_id in votes.index else 'not voting'
                if vote_i == vote_j and vote_i != 'not voting':
                    adjancency[i, j] = 1
    
    return adjancency

In [35]:
# all_adjacency = {}

for i in range(0, len(all_laws)):
    row = all_laws.iloc[i]
    url = row['vote_uri']
    keyname = '_'.join(url.split('/')[4:]).split('.')[0]

    if keyname not in all_adjacency:
        print(keyname)
        congress = row['congress']
        members = all_members[all_members['congress'] == congress].sort_values(['party', 'id'])
        members = members.set_index('id')
        req = requests.get(url, headers={'X-API-Key': API_KEY}).json()
        results = pd.DataFrame(req['results']['votes']['vote']['positions'])
        results = results.set_index('member_id')
    
        all_adjacency[keyname] = get_adjacency(members.index, results)

    else:
        continue

In [33]:
with open(DATA_PATH + '/all_adjacency.pickle', 'wb') as handle:
    pickle.dump(all_adjacency, handle, protocol=pickle.HIGHEST_PROTOCOL)

# del all_adjancency

In [206]:
with open(DATA_PATH + '/all_adjacency.pickle', 'rb') as handle:
    all_adjacency = pickle.load(handle)

In [209]:
len(all_adjancency.keys())

6834

In [43]:
all_bills = []

for i in range(0, len(all_laws)):
    row = all_laws.iloc[i]
    url = row['bill_api_uri']
    keyname = '_'.join(row['vote_uri'].split('/')[4:]).split('.')[0]

    if keyname not in all_bills and url is not np.nan:
        d = {}
        
        print(keyname)
        try:
#             req = requests.get(url, headers={'X-API-Key': API_KEY}).json()
            results = req['results'][0]
            d['law_id'] = keyname
            d['sponsor_party'] = results['sponsor_party']
            d['sponsor_id'] = results['sponsor_id']
            d['primary_subject'] = results['primary_subject']
            d['committees'] = results['committees']
            d['short_title'] = results['short_title']
    
            all_bills.append(d)
        except:
            pass

    else:
        continue

In [42]:
pd.DataFrame(all_bills).to_csv(DATA_PATH + '/bills_20.years.csv')